# The Dataset

The dataset is based on a similar dataset that was [published together with other datasets from fundamental physics](https://doi.org/10.1007/s41781-022-00082-6).

Here we want to use a slighty less processed version to learn a bit about the physics content and how we can transform it such that it can be input to a neural network.

It is provided in [Parquet](https://parquet.apache.org) format - written out using the [Awkward Array](https://awkward-array.org) library.

In [ ]:
#!pip install "awkward>=2"
# if not installed yet

In [ ]:
import awkward as ak
import numpy as np

The dataset contains 4 "row groups" - each of them containing 100k events. Here we load only the first row group (index 0):

In [ ]:
from urllib.request import urlretrieve
import os

In [ ]:
filename = "smartbkg_dataset_4k.parquet"
url = "https://zenodo.org/records/15303496/files/smartbkg_dataset_4k_training.parquet?download=1"

In [ ]:
if not os.path.exists(filename):
    urlretrieve(url, filename)

In [ ]:
ak_data = ak.from_parquet(filename, row_groups=[0])

In [ ]:
ak_data

The structure of the data is the following
- `particles`: generator-level quantities for each event - a variable length list of particles to be used as input for the training
    - `pdg`: a number identifying the particle type according to the [numbering scheme of the Particle Data Group (PDG)](https://pdg.lbl.gov/2022/reviews/rpp2022-rev-monte-carlo-numbering.pdf)
    - `index`: an identifier of the particle in a particular event
    - `mother_index`: the `index` of the mother particle - this defines the decay tree
    - the rest are generator-level features of the particles (all in the lab frame)
        - `prodTime`: time of production (e.g. decay of mother particle) in ns
        - `x, y, z`: positions of the production vertex in cm
        - `px, py, pz, energy`: 4-momentum vector components in GeV (natural units with c=1)
- `label`: `1` for events that **pass** the downstream event selection and `0` for those that **fail** it

Here we will use a pandas DataFrame representation:

In [ ]:
import pandas as pd

In [ ]:
df_particles = ak.to_dataframe(ak_data.particles, levelname=lambda i: {0: "event", 1: "particle"}[i])
df_particles

In [ ]:
labels = ak_data.label.to_numpy()

In [ ]:
df_label = pd.DataFrame(labels, columns=["label"])
df_label.index = df_label.index.rename("event")
df_label

In [ ]:
df = df_particles.join(df_label)
df

First, let's plot the global distribution (across all events) for our particle features:

In [ ]:
feature_columns = ["prodTime", "x", "y", "z", "energy", "px", "py", "pz"]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_feats(df, column_names, bins=100, log=True, range_fn=None):
    fig, axs = plt.subplots(ncols=4, nrows=2, figsize=(15, 5))
    for ax, field in zip(axs.ravel(), column_names):
        array = df[field].to_numpy()
        if range_fn is None:
            bin_range = None
        else:
            bin_range = range_fn(array)
        ax.hist(array, bins=bins, range=bin_range)
        if log:
            ax.set_yscale("log")
        ax.set_title(field)
    fig.tight_layout()
    plt.close(fig)
    return fig

In [ ]:
plot_feats(df, feature_columns, log=True) # log scale, full range

In [ ]:
plot_feats(df, feature_columns, log=False, range_fn=lambda x: np.quantile(x, [0.05, 0.95])) # linear scale, 5% - 95% quantiles of distribution

We can see a few characteristics of the experiment that have been taken into account in the simulation:
- A peak in the energy at the total energy of colliding beams (7 GeV + 4 GeV = 11 GeV) - this corresponds to the Y(4S) resonance particles.
- The next-lower peak in the energy is at around half of that - corresponding to the B mesons. The rest are then all the other particles in the decay chains.
- The `z` vertex positions and momenta have a bias towards positive values (and a peak at 7 GeV - 4 GeV = 3 GeV for `pz` for the Y(4S) particles) - due to the asymmetric beam energies
- The peak in `px` at around 0.46 GeV comes from the crossing angle of the beams - resulting in a small boost in x-direction for the Y(4S) particles.

Note: particles with `x`, `y`, `z` values outside of the `[-10, 10]` range are removed for technical reasons.

<div class="alert alert-block alert-success">
    <b>Exercise:</b> Overlay the distribution of features for events with label 0 and 1. Does the global distribution of features alone already provide discriminative power?
<br><br>

Hint: use e.g. `array[df.label==0]` to get the global distribution for label 0 events. To overlay both histograms you can use `histtype="step"` or `alpha=0.5` as arguments to `plt.hist`.
</div>

We can build decay trees using the `index` and `mother_index` fields. For visualization we use `grahviz` and to convert the PDG ids we use a dictionary to convert into a human readable unicode string:

In [ ]:
from pdg_to_unicode import pdg_to_unicode

The first 3 particles of each event are typically the Y(4S) resonance `300553` decaying into a neutral B and anti-B meson:

In [ ]:
df.pdg.loc[0][:3]

In [ ]:
pdg_to_unicode[300553], pdg_to_unicode[-511], pdg_to_unicode[511]

In [ ]:
import graphviz

In [ ]:
def draw_graph(x):
    g = graphviz.Digraph()
    for i, pdg in zip(x["index"], x["pdg"]):
        g.node(str(i), label=pdg_to_unicode[pdg])
    for src, dst in zip(x["mother_index"], x["index"]):
        if src == dst: # particles without mother point to themselves
            continue
        g.edge(str(src), str(dst))
    return g

In [ ]:
event_id = 0 # change this to look at different events
draw_graph(df.loc[event_id])

So each one of these decay trees (together with the features for each particle) will make up **one instance** of our training dataset.

## Preprocessing

To use the data as input for a neural network we need to do some preprocessing.

### Index the PDG ids
First, we need to find a way to input the pdg ids.

The numerical values are not very useful for processing in neural network layers, so we want to convert the particle identifier into a vector (compare e.g. word embeddings in a language model) - either with [one-hot encoding](https://en.wikipedia.org/wiki/One-hot#Machine_learning_and_statistics) or by utilizing an [Embedding layer](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html). In any case, we need to index (enumerate) them such that they are numbers in the range of `[0, num_pdg_ids]`

In [ ]:
unique_pdg_ids = np.unique(df.pdg.to_numpy())
unique_pdg_ids

In [ ]:
num_pdg_ids = len(unique_pdg_ids)

In [ ]:
num_pdg_ids

We will use the following dictionary to map the pdg ids. We will start counting at 1 since 0 will be a special padding value (more later)

The value `num_pdg_ids + 1` will be reserved as a fallback token in case we encounter unseen pdg ids in the test data.

In [ ]:
mapping = dict(zip(unique_pdg_ids.tolist(), range(1, num_pdg_ids + 1)))

for (key, value), _ in zip(mapping.items(), range(10)):
    print(f"{key}: {value}")
print("...")

In [ ]:
def map_np(array, mapping, fallback=None):
    """
    Apply a mapping over a numpy array - along the lines of
    https://stackoverflow.com/a/16993364
    """
    if fallback is None:
        fallback = max(mapping.values()) + 1
    # inv is the original array with the values replaced by their indices in the unique array
    unique, inv = np.unique(array, return_inverse=True)
    np_mapping = np.array([mapping.get(x, fallback) for x in unique])
    return np_mapping[inv]

In [ ]:
# Example:
map_np(np.array([42, 753, 42, 1111, 753, 86277, 27786]), {42: 1, 753: 2, 1111: 3}, fallback=4)

To have a consistent mapping for all datasets we will use the mapping defined in `pdg_mapping.json` which was produced by the script `create_pdg_mapping.py` that ran on all 4 row groups of the training data.

In [ ]:
import json

In [ ]:
with open("pdg_mapping.json") as f:
    pdg_mapping = dict(json.load(f))

In [ ]:
# adds another array with the mapped particle ids to the DataFrame
df["pdg_mapped"] = map_np(df.pdg, pdg_mapping, fallback=len(pdg_mapping) + 1)

In [ ]:
df.pdg

In [ ]:
df.pdg_mapped

### List of arrays representation

For loading the data into ML models it is useful to also have a representation of the data as a list of numpy arrays. We will have one list for each `pdg_mapped`, `index` and `mother_index` as well as one list of 2D numpy arrays for the particle features.

To create this we will proceed as follows:
* create "flat" numpy arrays (single array across event boundaries)
* use a `pd.DataFrame.groupby` operation to get the indices of groups of particles
* use these indices to create numpy arrays for each event

In [ ]:
flat = {
    "features": df[feature_columns].to_numpy(),
    "pdg_mapped": df["pdg_mapped"].to_numpy(),
    "index": df["index"].to_numpy(),
    "mother": df["mother_index"].to_numpy(),
}

In [ ]:
gb_indices = df.groupby("event").indices

e.g. the following will give the indices of the first event:

In [ ]:
gb_indices[0]

Now we can fill the list of arrays:

In [ ]:
data = {}
for idx in gb_indices.values():
    for k, array in flat.items():
        data.setdefault(k, [])
        data[k].append(array[idx])

In [ ]:
data["features"][0].shape

In [ ]:
data["pdg_mapped"][0].shape

# A simple "Deep Set" Model

As a starting point we will view our data as an **unordered set** of particles. The *Deep Set* model we will use applies a per **per-item transformation** ($\phi$) followed by a **permutation invariant aggregation**, typically taking the sum/mean or min/max whose output can then be transformed ($\rho$) by any means, e.g. another MLP.

![](figures/deep_set_transformation.png)

See [arXiv:1703.06114](https://arxiv.org/abs/1703.06114) for a detailed discussion.

The per-item transformation we can do easily do in `torch` by using a `Linear` layer - when operating on a sequence/set it will be applied per item:

In [ ]:
import torch
from torch import nn

In [ ]:
per_item_layer = nn.Linear(in_features=3, out_features=4)

Let's create some example inputs to see what happens to them when they are passed through the layers:

In [ ]:
inputs = torch.rand(2, 5, 3)
inputs

In [ ]:
per_item_layer(inputs)

For aggregation we just take the mean:

In [ ]:
inputs.mean(axis=1)

A possible model (operating only on particle features) would be:

In [ ]:
class DeepSet(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.per_item_mlp = nn.Sequential(
            nn.Linear(in_features, 32),
            nn.ReLU(),
        )
        self.global_mlp = nn.Sequential(
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )
        
    def forward(self, inputs):
        x = inputs
        x = self.per_item_mlp(x)
        x = x.mean(axis=-2)
        x = self.global_mlp(x)
        return x

model = DeepSet(len(feature_columns))
model

The model can take arbitrary sized batches with an arbitrary sized set of features:

So this maps a set of particle features into a single number:

In [ ]:
inputs = torch.tensor(data["features"][0][np.newaxis, :])
inputs.shape

In [ ]:
model(inputs)

But what if not all sets in a batch of events have the same size?

## 0-Padding

Many standard operations in NN frameworks like PyTorch work only on arrays with same length lists for each subentry (there are also implementations for sparse computations for neural networks, but we won't consider these here). So we need to have **sequences of the same length, within one batch of data**.

To achieve this we will **fill the values with 0** for instances in a batch with a lower number of particles

We can pad the sequences with the following helper function (similar to [`torch.nn.utils.rnn.pad_sequence`](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html), but using numpy). To do this, we first create a matrix of zeros with the appropriate shape and data type. Then we replace the leftmost zeros row by row with the existing sequences. Due to the way numpy arrays work, this is much more efficient than concatenating each sequence with arrays of zeros and stacking them.

In [ ]:
def pad_sequences(sequences, maxlen=None):
    if maxlen is None:
        maxlen = max(len(array) for array in sequences)
    if sequences[0].ndim == 2:
        shape = (len(sequences), maxlen, sequences[0].shape[-1])
    else:
        shape = (len(sequences), maxlen)
    batch = np.zeros(shape, dtype=sequences[0].dtype)
    for i, array in enumerate(sequences):
        batch[i, :len(array)] = array
    return batch

In [ ]:
pad_sequences(data["pdg_mapped"][:5]).shape

In [ ]:
pad_sequences(data["pdg_mapped"][:5], maxlen=100).shape

In [ ]:
pad_sequences(data["pdg_mapped"][:5])

In [ ]:
pad_sequences(data["features"][:5]).shape

In [ ]:
pad_sequences(data["features"][:5])

## Masking

A typical convention to treat values that are supposed to be ignored is to propagate a mask array through. In pytorch the convention is usually that values that are supposed to be ignored have a `True` in the mask and those that are not supposed to be ignored a `False`.

In [ ]:
test_batch = np.zeros((2, 5, 4))
test_batch[0, :3] = np.random.rand(3, 4)
test_batch[1, :4] = np.random.rand(4, 4)
test_batch = torch.tensor(test_batch)

In [ ]:
test_batch

In [ ]:
mask = (test_batch == 0).all(axis=-1)

In [ ]:
mask

The mask can be inverted using `~`

In [ ]:
~mask

We can compute a masked average by setting the masked values to 0 and then taking first the sum, followed by dividing my the sum of the inverse mask.

In [ ]:
def masked_average(batch, mask):
    batch = batch.masked_fill(mask[..., np.newaxis], 0)
    sizes = (~mask).sum(axis=1, keepdim=True)
    return batch.sum(axis=1) / sizes

Breaking this down step by step:

* `batch.masked_fill(mask[..., np.newaxis], 0)` fill masked values with 0s
* `[..., np.newaxis]` adds another dimension to ensure the same number of dimensions as the batch
* `~mask` invert the mask
* `(~mask).sum(axis=1, keepdim=True)` summing over not-masked values produces the sizes for each batch element. The argument `keepdim=True` ensures to keep the same number of dimensions as the batch.
* `batch.sum(axis=1) / sizes` produces the average

In [ ]:
masked_average(test_batch, mask)

In [ ]:
test_batch.sum(axis=1) / torch.tensor([[3], [4]]) # works in this case since masked values are already 0

In [ ]:
test_batch.mean(axis=1) # not the same, since it averages over all 5 elements!

## Fit the model

Putting everything together we can build a model as follows:
- mask 0-padded entries
- apply a the per-item transformation as a single `Linear` layer
- calculate the mean across the sequence of particles
- apply a number of global `Linear` layers on the averaged features
- output a single number representing the probability of our `y` labels

In [ ]:
class DeepSet(nn.Module):
    def __init__(self, num_features=8, units=32):
        super().__init__()
        self.per_item_mlp = nn.Sequential(
            nn.Linear(num_features, units),
            nn.ReLU(),
        )
        self.global_mlp = nn.Sequential(
            nn.Linear(units, units),
            nn.ReLU(),
            nn.Linear(units, 1)
        )
        
    def forward(self, inputs, mask=None):
        x = inputs
        x = self.per_item_mlp(x)
        if mask is not None:
            x = masked_average(x, mask)
        else:
            x = x.mean(axis=-2)
        x = self.global_mlp(x)
        return x
    
model = DeepSet()
model

We now fit the model using a random 10% fraction of the dataset for validation during training:

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(data["features"], labels, test_size=0.1, shuffle=True)

We will create a `Dataset` that inherits from `torch.utils.data.Dataset` that will provide `x` (input) and `y` (target) pairs:

In [ ]:
from torch.nn import functional as F
from torch.utils.data import DataLoader

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.x)
        
    def __getitem__(self, i):
        return self.x[i], self.y[i]

In [ ]:
ds_train = Dataset(x_train, y_train)
ds_val = Dataset(x_val, y_val)

In [ ]:
x, y = ds_train[0]
x.shape

In [ ]:
y

We will then use a `DataLoader` to put these instances into batches. We have to provide a function that applies the `pad_sequences` to our input features and calculates the mask then as the `collate_fn` argument to our `DataLoader`:

In [ ]:
def collate_fn(inputs):
    x = [i[0] for i in inputs]
    y = [i[1] for i in inputs]
    x = torch.tensor(pad_sequences(x))
    y = torch.tensor(y)
    mask = (x == 0).all(axis=-1)
    return x, y, mask

In [ ]:
for batch in DataLoader(ds_train, batch_size=256, collate_fn=collate_fn, shuffle=True):
    pass

In [ ]:
batch

Now we can implement the training loop:

For the loss we will use the binary cross entropy - `with_logits` means we use outputs without a sigmoid activation function applied.

The activation function is applied in the loss function instead, allowing a more numerically stable computation:

In [ ]:
def loss_fn(logits, y):
    return F.binary_cross_entropy_with_logits(logits.squeeze(), y.float())

We also want to track the accuracy - the fraction of correctly labelled events taking the most likely label

In [ ]:
def accuracy_fn(logits, y):
    return (logits.squeeze().sigmoid().round() == y).float().mean()

Now we implement the training loop. We use the adam optimizer with default parameters for now:

In [ ]:
def fit(model, dl_train, dl_val, epochs=10, device="cpu", history=None):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters())

    def train_step(x, y, mask):
        model.train()
        optimizer.zero_grad()
        logits = model(x, mask=mask)
        loss = loss_fn(logits, y)
        loss.backward()
        optimizer.step()
        return logits.detach().cpu(), loss.detach().cpu()

    def test_step(x, y, mask):
        model.eval()
        with torch.no_grad():
            logits = model(x, mask=mask)
            return logits.cpu(), loss_fn(logits, y.to(device)).cpu()

    def to_device(x, y, mask):
        if isinstance(x, dict):
            x = {k: v.to(device) for k, v in x.items()}
        else:
            x = x.to(device)
        y = y.to(device)
        mask = mask.to(device)
        return x, y , mask

    if history is None:
        history = []
    for epoch in range(epochs):
        print(f"Epoch {epoch}")
        train_loss = []
        train_acc = []
        val_loss = []
        val_acc = []
        for i, (x, y, mask) in enumerate(dl_train):
            x, y, mask = to_device(x, y, mask)
            logits, loss = train_step(x, y, mask)
            train_loss.append(float(loss))
            train_acc.append(float(accuracy_fn(logits, y)))
            print(
                f"Batch {i:03d}/{len(dl_train)}, "
                f"Train loss: {np.mean(train_loss):.3f}, "
                f"Train accuracy: {np.mean(train_acc):.3f}",
                end="\r" if i != len(dl_train) - 1 else ", ",
                flush=True,
            )
        for x, y, mask in dl_val:
            x, y, mask = to_device(x, y, mask)
            logits, loss = test_step(x, y, mask)
            val_loss.append(float(loss))
            val_acc.append(float(accuracy_fn(logits, y)))
        print(
            f"Validation loss: {np.mean(val_loss):.3f}, "
            f"Validation accuracy: {np.mean(val_acc):.3f}"
        )
        history.append(
            {
                "loss": np.mean(train_loss),
                "val_loss": np.mean(val_loss),
                "acc": np.mean(train_acc),
                "val_acc": np.mean(val_acc),
            }
        )
    return history

In [ ]:
history = []

dl_opts = dict(batch_size=256, collate_fn=collate_fn)
dl_train = DataLoader(ds_train, shuffle=True, **dl_opts)
dl_val = DataLoader(ds_val, **dl_opts)

In [ ]:
history = fit(model, dl_train, dl_val, history=history)

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(history).plot()

# Embedding layers and multiple inputs

So far we have not used the `pdg` field - the particle type information. One way to use such categorical features is to feed them through an [`Embedding`](https://keras.io/api/layers/core_layers/embedding/) layer.

Since we have mapped the PDG ids to numbers in a continuous range we can directly use such a layer - remember that we shifted the numbers by 1 to be able to use 0 as a padding value. The number of output dimensions is a hyperparameter of this layer:

In [ ]:
embed_dim = 8
embedding = nn.Embedding(num_pdg_ids + 1, embed_dim)

All this layer does is to have a learnable matrix of size `(num_categories, embed_dim)` that maps each category to a vector of fixed size:

In [ ]:
embedding(torch.tensor(1))

In [ ]:
embedding(torch.tensor(2))

It essentially just picks the row with the specified index:

In [ ]:
embedding.weight.shape

In [ ]:
embedding.weight[1] == embedding(torch.tensor(1))

In [ ]:
embedding.weight[2] == embedding(torch.tensor(2))

This is equivalent to applying a Dense layer to one-hot encoded categorical features.

To use both PDG ids and the rest of the particle features as inputs you can create a model that takes multiple inputs and then concatenate the embedded PDG ids with the other features, e.g.

In [ ]:
class CombinedModel(nn.Module):
    def __init__(self, num_feat=8, embed_dim=8, num_pdg_ids=num_pdg_ids, units=32):
        super().__init__()
        self.embedding = nn.Embedding(num_pdg_ids + 1, embed_dim)
        self.deep_set = DeepSet(num_features=num_feat + embed_dim, units=units)

    def forward(self, inputs, mask=None):
        pdg = inputs["pdg"]
        feat = inputs["feat"]
        emb = self.embedding(pdg)
        x = torch.cat([feat, emb], -1)
        return self.deep_set(x, mask=mask)

In [ ]:
combined_model = CombinedModel()
combined_model

In [ ]:
torch.tensor(data["features"][0]).shape

In [ ]:
torch.tensor(data["features"][0]).unsqueeze(0).shape # alternative to [np.newaxis, :]

In [ ]:
inputs = dict(
    pdg=torch.tensor(data["pdg_mapped"][0]).unsqueeze(0),
    feat=torch.tensor(data["features"][0]).unsqueeze(0)
)

In [ ]:
combined_model(inputs)

Note: When fitting such a model you need to adjust the `Dataset` and the `collate_fn` to provide the inputs as a dictionary with fields `"pdg"` and `"feat"`

# Graph Network

Another thing we haven't used yet is the graph structure of the events (the particles form a decay tree). One way to incorporate this is via Graph Convolutions.

## Graph Convolutions

Similar to convolutional networks where we update the state of each pixel by aggregating over neigboring pixels we can perform a *graph convolution* by aggregating over neighboring nodes in a graph:

![cnn vs gcn](figures/cnn_vs_gcn.jpg)

(figure from https://zhuanlan.zhihu.com/p/51990489)

In the "Deep sets" language such a graph convolution corresponds to a *permutation equivariant* tranformation of the set of nodes, since it also does not depend on the ordering if the aggregation is done in a permutation invariant way (e.g. sum/mean/min/max).

A rather simple implementation is given by the update rule introduced in [arXiv:1609.02907](https://arxiv.org/abs/1609.02907) ("GCN")

$ H^{(l+1)} = \sigma(\tilde{D}^{-\frac{1}{2}}\tilde{A}\tilde{D}^{-\frac{1}{2}}H^{(l)}W^{(l)}) $

where $A$ is the *adjacency matrix*, $D$ the *degree matrix*,  $H^{(l)}$ the hidden state of layer $l$ and $W^{(l)}$ the weight matrix of the layer $l$. The tilde above $A$ and $D$ indicates that self-loops were added (all nodes are neighbors of themselves).

An equivalent formulation is

$ h_i^{(l+1)} = \sigma\left(\sum\limits_{j\in\mathcal{N}(i)}\frac{1}{c_{ij}}h^{(l)}_j W^{(l)}\right) $

where $ \mathcal{N(i)} $ is the set of neighbors of node $i$ and $c_{ij} = \sqrt{N_i}\sqrt{N_j}$ with $N_i$ being the number of neigbors of node $i$

<div class="alert alert-block alert-success">
    <b>Exercise:</b> Verify for one example event that the matrix multiplication of the adjacency matrix with the feature matrix is equivalent to taking the sum over neighbor features for each node. In other words that
    
$ (AF)_{ij} = \sum\limits_{k\in\mathcal{N}(i)}F_{kj} $
</div>

e.g with

In [ ]:
F = np.array([[1, 2], [3, 4], [5, 6]])
F # 3 nodes, 2 features each

In [ ]:
A = np.array([[1, 1, 0], [1, 1, 1], [0, 1, 1]])
A # all nodes are neighbors to themselves, node1 is neighbor of node2 and vice versa, node2 is neighbor of node3 and vice versa

We can decompose the GCN into an operation that normalizes the adjacency matrix via the node degrees:

In [ ]:
def normalize_adjacency(adj):
    deg_diag = adj.sum(axis=2)
    deg12_diag = torch.where(deg_diag != 0, deg_diag**-0.5, 0)
    # normalization coefficients are outer product of inverse square root of degree vector
    # gives coeffs_ij = 1 / sqrt(N_i) / sqrt(N_j)
    coeffs = deg12_diag[:, :, np.newaxis] @ deg12_diag[:, np.newaxis, :]
    return adj.float() * coeffs

In [ ]:
normalize_adjacency(torch.tensor(A)[np.newaxis, :])

and the update rule that takes the node inputs and the adjacency matrix as parameters:

In [ ]:
class GCN(nn.Module):
    """
    Simple graph convolution. Equivalent to GCN from Kipf & Welling (https://arxiv.org/abs/1609.02907)
    when fed a normalized adjacency matrix.
    """
    def __init__(self, in_features, units):
        super().__init__()
        self.linear = nn.Linear(in_features, units)

    def forward(self, inputs, adjacency):
        return adjacency @ self.linear(inputs)

## Adjacency matrix

To get the adjacency matrices in our dataset we create need to create a matrix where the `index` is equal to the `mother` field, e.g. for

In [ ]:
index = np.array([1, 2, 3])
mother = np.array([1, 1, 2])

we do 

In [ ]:
index[:, np.newaxis] == mother[np.newaxis, :]

we used slicing with `np.newaxis` to add extra dimensions to the array, so we made a comparison between a column vector and a row vector which numpy automatically broadcasts

In [ ]:
index[:, np.newaxis], mother[np.newaxis, :]

We will now create adjacency matrices for all events:

In [ ]:
def get_adj(index, mother):
    return (
        (mother[np.newaxis, :] == index[:, np.newaxis]) # mother-daughter relations
        | (index[np.newaxis, :] == mother[:, np.newaxis]) # daughter-mother relations
        | (index[np.newaxis, :] == index[:, np.newaxis]) # self loops
    )

In [ ]:
data["adj"] = [get_adj(index, mother) for index, mother in zip(data["index"], data["mother"])]

In [ ]:
plt.imshow(data["adj"][0], cmap="Greys")

This translates in the same graphs we have seen before, but now we have both mother-daughter and daughter-mother connections and also a connection from each node to itself:

In [ ]:
def draw_graph_from_adjacency(adj, pdg):
    g = graphviz.Digraph()
    for i, pdg_i in enumerate(pdg):
        g.node(str(i), label=pdg_to_unicode[pdg_i])
    for src in range(len(pdg)):
        for dst in range(len(pdg)):
            if adj[src][dst]:
                g.edge(str(src), str(dst))
    return g

In [ ]:
event_index = 0 # change to look at different events
draw_graph_from_adjacency(data["adj"][event_index], df.loc[event_index].pdg)

When iterating over the training dataset later to fit a model we also need to create batches of adjacency matrices.

Here we also need to pad the batches of adjacency matrices to the maximum event length:

In [ ]:
def pad_adjacencies(adj_list):
    maxlen = max(len(adj) for adj in adj_list)
    batch = np.zeros((len(adj_list), maxlen, maxlen), dtype=bool)
    for i, adj in enumerate(adj_list):
        batch[i, :len(adj), :len(adj)] = adj
    return batch

So each batch will have features, pdg ids and adjacency matrices:

In [ ]:
pad_sequences(data["pdg_mapped"][:256]).shape

In [ ]:
pad_sequences(data["features"][:256]).shape

In [ ]:
pad_adjacencies(data["adj"][:256]).shape

Putting it together a `Dataset` and `collate_fn` for the `DataLoader` for this could look like the following

In [ ]:
class GraphDataset(torch.utils.data.Dataset):
    def __init__(self, feat, pdg, adj, y):
        self.feat = feat
        self.pdg = pdg
        self.adj = adj
        self.y = y
        
    def __len__(self):
        return len(self.feat)
    
    def __getitem__(self, i):
        x = {
            "feat": self.feat[i],
            "pdg": self.pdg[i],
            "adj": self.adj[i]
        }
        y = self.y[i]
        return x, y

In [ ]:
def collate_fn_graphs(inputs):
    feat, pdg, adj = [
        [x[key] for x, y in inputs] for key in ["feat", "pdg", "adj"]
    ]
    y = [y for x, y in inputs]
    x = {
        "feat": torch.tensor(pad_sequences(feat)),
        "pdg": torch.tensor(pad_sequences(pdg)),
        "adj": torch.tensor(pad_adjacencies(adj)),
    }
    y = torch.tensor(y)
    mask = (x["feat"] == 0).all(axis=-1)
    return x, y, mask

In [ ]:
dl = DataLoader(
    GraphDataset(feat=data["features"], pdg=data["pdg_mapped"], adj=data["adj"], y=labels),
    batch_size=256,
    shuffle=True,
    collate_fn=collate_fn_graphs,
)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
for batch in tqdm(dl):
    pass

In [ ]:
x, y, mask = batch

In [ ]:
x["feat"].shape

In [ ]:
x["pdg"].shape

In [ ]:
x["adj"].shape

In [ ]:
y.shape

In [ ]:
mask.shape

For using `GCN` layers in a model one needs the adjacency matrices as an additional input and feed them through `normalize_adjaceny` once before passing them over to a `GCN` layer. Here an example for a torch model that only applies a single `GCN` layer:

In [ ]:
class GraphNetwork(nn.Module):
    def __init__(self, in_features, units=32):
        super().__init__()
        self.gcn = GCN(in_features, units)

    def forward(self, inputs, mask=None):
        adj = inputs["adj"]
        feat = inputs["feat"]
        adj = normalize_adjacency(adj)
        return self.gcn(feat, adj)

In [ ]:
graph_network = GraphNetwork(len(feature_columns))

In [ ]:
out = graph_network(x)
out.shape